In [23]:
import pandas as pd

In [24]:
bitcoin_prices_df = pd.read_parquet('../data/bitcoin_prices.parquet.gzip')
reddit_titles_df = pd.read_parquet('../data/reddit_titles.parquet.gzip')
sentiment_scores_df = pd.read_parquet('../data/sentiment_scores.parquet.gzip')

In [11]:
sentiment_scores_df = sentiment_scores_df.rename(columns={
    'date': 'orig_date',
    'offset_date': 'date'
})

In [12]:
sentiment_scores_df

,orig_date,compound,date
0,2021-03-15 03:00:00,0.039934,2021-03-15 06:00:00
1,2021-03-15 06:00:00,0.007725,2021-03-15 09:00:00
2,2021-03-15 09:00:00,0.081969,2021-03-15 12:00:00
3,2021-03-15 12:00:00,0.124559,2021-03-15 15:00:00
4,2021-03-15 15:00:00,0.071304,2021-03-15 18:00:00
...,...,...,...
236,2021-04-13 15:00:00,NaN,2021-04-13 18:00:00
237,2021-04-13 18:00:00,NaN,2021-04-13 21:00:00
238,2021-04-13 21:00:00,NaN,2021-04-14 00:00:00
239,2021-04-14 00:00:00,NaN,2021-04-14 03:00:00


In [13]:
bitcoin_prices_df['price_diff_lag_1'] = bitcoin_prices_df['price_diff'].shift(1)
bitcoin_prices_df['price_diff_lag_2'] = bitcoin_prices_df['price_diff'].shift(2)
bitcoin_prices_df['price_diff_lag_3'] = bitcoin_prices_df['price_diff'].shift(3)

In [14]:
bitcoin_prices_df = bitcoin_prices_df.reset_index(drop=True)

In [15]:
bitcoin_prices_df['day'] = bitcoin_prices_df['date'].dt.day

In [16]:
bitcoin_prices_df['hour'] = bitcoin_prices_df['date'].dt.hour

In [17]:
everything = bitcoin_prices_df.join(sentiment_scores_df.set_index('date'), on='date')

In [18]:
everything = everything.dropna()

In [19]:
features_df = everything[['date', 'price_diff_lag_1', 'price_diff_lag_2', 'price_diff_lag_3', 'day', 'hour', 'compound']]
target_df = everything[['price_diff', 'date']]

In [20]:
everything = features_df.join(target_df.set_index('date'), on='date')

In [21]:
everything.to_parquet('../data/everything_v1.parquet.gzip', compression='gzip')